In [1]:
import geopandas
import pandas as pd
import glob
import shapefile
from tqdm import tqdm
import numpy as np
import pickle
import xlrd3
import os
import glob
import openpyxl

In [2]:
# read poi and locations from shp files
poi_file = '/home/yangwenhao/local/project/weibo_filter/app/data/POI_fid.xlsx'
full_names = pd.read_excel(poi_file, sheet_name=0)

In [3]:
class location(object):
    def __init__(self, fid, x, y):
        self.fid = fid
        self.x = x
        self.y = y

In [5]:
if os.path.exists('names2loc2_fid.pickle'):
    with open('names2loc2_fid.pickle', 'rb') as f:
        names2loc = pickle.load(f)
else:
    names2loc = {}
    for x in full_names[['FID', '名称', '经度', '纬度']].iterrows():
        # print(x)
        loc = location(fid=x[1].FID, x=x[1].经度, y=x[1].纬度)
        names2loc.setdefault(x[1].名称, []).extend([loc])

    with open('names2loc2_fid.pickle', 'wb') as f:
        pickle.dump(names2loc, f)

In [ ]:
# create place2index files giving tweets and locations
# pickle file: [places, sparse_matirx]

# sub_names = ['商业娱乐', '文化展览', '科研教育', '体育健身', '住区', '旅游景点', '企业办公', '公共服务', '交通设施', '医养设施']
for sht_idx in range(1, 11):
    full_names = pd.read_excel(poi_file, sheet_name=sht_idx)
    names = full_names.名称
    location_names = names
    numofprint = 4

    for lst in ['list2_img', 'list2_woimg']:
        text_csv = 'all_list/' + '%s.csv'%(lst)
        if not os.path.exists(text_csv):
            data = xlrd3.open_workbook('/home/yangwenhao/local/project/weibo_filter/app/data/' + lst + '.xlsx')
            table = data.sheet_by_index(0)#.sheets()[0]          #通过索引顺序获取
            # get full-text tweets
            texts = []
            if lst == 'first_2w':
                test_idx = 5
            elif lst == 'list2_img':
                test_idx = 4
            elif lst == 'list2_woimg':
                test_idx = 6
            else:
                test_idx = 6

            for i in range(1, table.nrows):
                texts.append(table.row(i)[test_idx].value)

            texts = pd.DataFrame(texts, columns=['text'])
            texts.to_csv('%s.csv'%(lst), index=False)
        else:
            texts = pd.read_csv(text_csv)
        # match all palces for all lines
        places,sparse_matirx = [],[]
        for idx, i in tqdm(enumerate(location_names.unique()), ncols=50,
                           total=len(location_names.unique())): #[297604:]
            try:
                result = texts.text.str.contains(i) # 12:26:26
                if result.to_numpy().sum() > 0:
                    places.append(i)
                    sparse_matirx.append(result.to_numpy())
            except Exception as e:
                continue

            if (idx+1) == len(location_names.unique()):
                print("出现的地点数: ", len(places), " 记录的列数:", len(sparse_matirx))
                with open('all_list3/%d_pla2idx_%s.pickle' % (sht_idx, lst), 'wb') as f:
                    pickle.dump([places, sparse_matirx], f)
                places,sparse_matirx = [],[]

In [11]:
def MatchProcess(poi_file, sht_idx, lst):
    full_names = pd.read_excel(poi_file, sheet_name=sht_idx)
    names = full_names.名称
    location_names = names
    
    text_csv = 'all_list/' + '%s.csv'%(lst)
    texts = pd.read_csv(text_csv)
    places,sparse_matirx = [],[]
    for idx, i in tqdm(enumerate(location_names.unique()), ncols=50,
                       total=len(location_names.unique())): #[297604:]
        try:
            result = texts.text.str.contains(i) # 12:26:26
            if result.to_numpy().sum() > 0:
                places.append(i)
                sparse_matirx.append(result.to_numpy())
        except Exception as e:
            continue

    print("sheet: ", sht_idx, " 出现的地点数: ", len(places), " 记录的列数:", len(sparse_matirx))
    with open('all_list3/%d_pla2idx_%s.pickle' % (sht_idx, lst), 'wb') as f:
        pickle.dump([places, sparse_matirx], f)

In [12]:
from multiprocessing import Pool
pool = Pool(processes=14)  # 创建nj个进程
# loc_csv = 'tianjin.csv'
lst = 'list2_img'
for sht_idx in range(1, 11):
    pool.apply_async(MatchProcess, args=(poi_file, sht_idx, lst))
pool.close()  # 关闭进程池，表示不能在往进程池中添加进程
pool.join()  # 等待进程池中的所有进程执行完毕，必须在close()之后调用

100%|███████████| 724/724 [01:15<00:00,  9.55it/s]


sheet:  2  出现的地点数:  161  记录的列数: 161


100%|█████████| 1121/1121 [02:03<00:00,  9.08it/s]


sheet:  8  出现的地点数:  100  记录的列数: 100


100%|█████████| 2590/2590 [04:27<00:00,  9.70it/s]


...

2680


100%|█████| 51038/51038 [1:09:15<00:00, 12.28it/s]


sheet:  7  出现的地点数:  1457  记录的列数: 1457


100%|███| 193378/193378 [3:47:23<00:00, 14.17it/s]


sheet:  1  出现的地点数:  7309  记录的列数: 7309


In [20]:
# save match results with pickle
headers = ["相关地点数量", "微博id", "微博主页", "文本", "日期", "图片", "转发数", "评论数", "点赞数"]
names2loc_key = set(list(names2loc.keys()))
sub_names = ['Business_Entertainment', 'Cultural_Exhibition', 'Education', 'Sports', 'Residential_District', 'Tourist_Attraction', 'Financial_Office', 'Public_Service', 'Transport', 'Medical_Facilities']

# for lst in ['list2_img', 'list2_woimg']:
for lst in ['list2_img']:
    text_csv = 'all_list/' + '%s.csv'%(lst)
    text_pickle = 'all_list3/' + '*_pla2idx_%s.pickle'%(lst)
    text_pickles = glob.glob(text_pickle)
    this_texts = pd.read_csv(text_csv, index_col=0)
    
    for t in text_pickles:
        append_cells = {}
        with open(t, 'rb') as f:
            [places, sparse_matirx] = pickle.load(f)
        places_np = np.array(places)
        sparse_np = np.array(sparse_matirx) #.shape
        if places_np.shape[0] > 0:
            for j, l in zip(this_texts.index, range(sparse_np.shape[1])):
                row_cells = []
                for i in places_np[np.where(sparse_np[:, l] == True)[0]]:
                    if i in names2loc_key:
                        i_xy = names2loc[i][0] # for new multi place lists
                        row_cells.append([i, i_xy.fid, i_xy.x, i_xy.y])
                    # else:
                    #     row_cells.append([i, "", ""])
                append_cells[j] = row_cells
        else:
            print('Skip %s ...'%(t))
            continue
            
        if lst == 'list2_img':
            read_cols = [2, 3, 4, 5, 6, 7, 8,  9]
        elif lst == 'list2_woimg':
            read_cols = [2, 3, 6, 7, 8, 9, 10, 11]

        read_lst = r'/home/yangwenhao/local/project/weibo_filter/app/data/' + lst + '.xlsx'
        title_name = sub_names[int(t.split('/')[1].split('_')[0])-1]
        # print(title_name, "...")
        save_lst = 'all_list3/' + title_name + '.xlsx' # t.replace('pickle', 'loc) 
        read_writebook = xlrd3.open_workbook(filename=read_lst)
        writebook = openpyxl.Workbook()
        # 添加sheet 写入excel, 参数对应 行, 列, 值
        sheet      = writebook.create_sheet(title=title_name, index=0)
        sht_row = 2 # openpyxl start from 1 instead of 0
        # 相关地点数量	微博id	微博主页	文本	日期	图片	转发数	评论数	点赞数
        for i, h in enumerate(headers):
            sheet.cell(1, i+1, h)

        # write header
        # find the tweet with the largest number of locations
        max_los = 1
        for i in append_cells:
            max_los = max(max_los, len(append_cells[i]))
            
        for i in range(1, max_los+1):
            sheet.cell(1, (i-1)*4+len(headers)+1, '地点提取%d'%i)
            sheet.cell(1, (i-1)*4+len(headers)+2, '地点FID%d'%i)
            sheet.cell(1, (i-1)*4+len(headers)+3, '经度%d'%i)
            sheet.cell(1, (i-1)*4+len(headers)+4, '纬度%d'%i)

        read_table = read_writebook.sheet_by_index(0)
        nrows = read_table.nrows
        assert (nrows-1) == len(append_cells), print("rows in excel: ", (nrows-1), "rows in loc_csv: ", len(append_cells))
        
        for i in tqdm(range(1, nrows), ncols=50):
            this_line = read_table.row_slice(i)
            this_locs = append_cells[i-1]
            locs_names = [l[0] for l in this_locs]

            if len(this_locs) > 0:
                this_row = sht_row
                sht_row += 1

                sheet.cell(this_row, 1, len(this_locs))     
                for j, c in enumerate(read_cols):
                    sheet.cell(this_row, j+2, this_line[c].value)

                for j, (name, fid, x, y) in enumerate(this_locs):
                    sheet.cell(this_row, j*4+len(headers)+1,   name)
                    sheet.cell(this_row, j*4+len(headers)+2,   fid)
                    sheet.cell(this_row, j*4+len(headers)+3,   x)
                    sheet.cell(this_row, j*4+len(headers)+4,   y)

        writebook.save(save_lst)
        print(save_lst, " completed!")

100%|█| 289992/289992 [00:01<00:00, 206590.36it/s]


all_list3/Public_Service.xlsx  completed!


100%|█| 289992/289992 [00:02<00:00, 104027.33it/s]


all_list3/Education.xlsx  completed!


100%|█| 289992/289992 [00:02<00:00, 117812.10it/s]


...

all_list3/Transport.xlsx  completed!


100%|█| 289992/289992 [00:01<00:00, 156650.19it/s]


all_list3/Tourist_Attraction.xlsx  completed!


100%|█| 289992/289992 [00:02<00:00, 123485.46it/s]


all_list3/Sports.xlsx  completed!


In [15]:
for n in names2loc_key:
    if len(n) == 1:
        print(n, end = ' ')

卝 嚟 

In [8]:
# save match results with pickle
headers = ["相关地点数量", "微博id", "微博主页", "文本", "日期", "图片", "转发数", "评论数", "点赞数"]
names2loc_key = set(list(names2loc.keys()))
sub_names = ['Business_Entertainment', 'Cultural_Exhibition', 'Education', 'Sports', 'Residential_District', 'Tourist_Attraction', 'Financial_Office', 'Public_Service', 'Transport', 'Medical_Facilities']

# for lst in ['list2_img', 'list2_woimg']:
for lst in ['list2_img']:
    text_csv = 'all_list/' + '%s.csv'%(lst)
    text_pickle = 'all_list3/' + '*_pla2idx_%s.pickle'%(lst)
    # text_pickles = glob.glob(text_pickle)
    text_pickles = ['all_list3/6_pla2idx_list2_img.pickle']
    
    this_texts = pd.read_csv(text_csv, index_col=0)
    
    for t in text_pickles:
        append_cells = {}
        with open(t, 'rb') as f:
            [places, sparse_matirx] = pickle.load(f)
        places_np = np.array(places)
        sparse_np = np.array(sparse_matirx) #.shape
        if places_np.shape[0] > 0:
            for j, l in zip(this_texts.index, range(sparse_np.shape[1])):
                row_cells = []
                for i in places_np[np.where(sparse_np[:, l] == True)[0]]:
                    if i in names2loc_key:
                        i_xy = names2loc[i][0] # for new multi place lists
                        row_cells.append([i, i_xy.fid, i_xy.x, i_xy.y])
                    # else:
                    #     row_cells.append([i, "", ""])
                append_cells[j] = row_cells
        else:
            print('Skip %s ...'%(t))
            continue
            
        if lst == 'list2_img':
            read_cols = [2, 3, 4, 5, 6, 7, 8,  9]
        elif lst == 'list2_woimg':
            read_cols = [2, 3, 6, 7, 8, 9, 10, 11]

        read_lst = r'/home/yangwenhao/local/project/weibo_filter/app/data/' + lst + '.xlsx'
        title_name = sub_names[int(t.split('/')[1].split('_')[0])-1]
        # print(title_name, "...")
        save_lst = 'all_list3/' + title_name + '_multiline.xlsx' # t.replace('pickle', 'loc) 
        read_writebook = xlrd3.open_workbook(filename=read_lst)
        writebook = openpyxl.Workbook()
        # 添加sheet 写入excel, 参数对应 行, 列, 值
        sheet      = writebook.create_sheet(title=title_name, index=0)
        sht_row = 2 # openpyxl start from 1 instead of 0
        # 相关地点数量	微博id	微博主页	文本	日期	图片	转发数	评论数	点赞数
        for i, h in enumerate(headers):
            sheet.cell(1, i+1, h)

        # write header
        # find the tweet with the largest number of locations
        # max_los = 1
        sheet.cell(1, len(headers)+1, '地点')
        sheet.cell(1, len(headers)+2, 'FID')
        sheet.cell(1, len(headers)+3, '经度')
        sheet.cell(1, len(headers)+4, '纬度')

        read_table = read_writebook.sheet_by_index(0)
        nrows = read_table.nrows
        assert (nrows-1) == len(append_cells), print("rows in excel: ", (nrows-1), "rows in loc_csv: ", len(append_cells))
        
        for i in tqdm(range(1, nrows), ncols=50):
            this_line = read_table.row_slice(i)
            this_locs = append_cells[i-1]
            locs_names = [l[0] for l in this_locs]

            if len(this_locs) > 0:
                this_row = sht_row

                for j, (name, fid, x, y) in enumerate(this_locs):
                    sheet.cell(this_row, 1, len(this_locs))     
                    for j, c in enumerate(read_cols):
                        sheet.cell(this_row, j+2, this_line[c].value)
                        
                    sheet.cell(this_row, len(headers)+1,   name)
                    sheet.cell(this_row, len(headers)+2,   fid)
                    sheet.cell(this_row, len(headers)+3,   x)
                    sheet.cell(this_row, len(headers)+4,   y)
                    
                    this_row += 1
                    sht_row = this_row
                    
                    
        writebook.save(save_lst)
        print(save_lst, " completed!")

100%|██| 289992/289992 [00:03<00:00, 84987.73it/s]


all_list3/Tourist_Attraction_multiline.xlsx  completed!
